/content/drive/MyDrive/Movie-Recommender-System-Project
/content/drive/MyDrive/Movie-Recommender-System-Project


In [3]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import pickle

In [4]:
#Reading Datasets
movies_org = pd.read_csv('tmdb_5000_movies.csv')
credits_org = pd.read_csv('tmdb_5000_credits.csv')

#Making duplicates to modify
movies = movies_org
credits = credits_org

In [5]:
#dmdb id (like imdb, id in that website)
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [ ]:
credits.head()

In [ ]:
# to get all the values of the first movies cast
credits.head(1)['crew'].values

In [6]:
#merging the two datasets
#mergin based on title because thats common for two datasets
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(1)

In [7]:
#Since we are making a content based recommender system, we have to create relevant tags for each movie
#Filter the columns based on whether they can be made to useful tags or not 
#eg budget is not important but genre is important

#important tags
#genre
#id - to get posters during app deployment 
#keywords
#title
#overview - to find content similarity
#cast 
#crew

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]


In [ ]:
#convert this into three columns: movie id, name and tags(overview,genres, keywords,cast,crew)
movies.isnull().sum()


In [8]:
#overview has 3 null values, which is less, so we can drop them
movies.dropna(inplace=True)

In [ ]:
#checking for duplicate values, if there is, we need to remove
movies.duplicated().sum()

In [9]:
#convert dataset from
#[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]
#to
#['Action','Adventure','Fantasy','Science Fiction']

def convert(object):
  list = []
  for i in ast.literal_eval(object) :
    list.append(i['name'])
  return list 

# i['name'] will give error: string indices must be integers, so we have to the '[...]' to [...]
#for this we use ast.literal_eval()


In [10]:
movies['genres']= movies['genres'].apply(convert)
#movies.head()
#ValueError malformed node or string occurs when you already converted the genres and rerun this cell


In [11]:
#same for keywords
movies['keywords'] = movies['keywords'].apply(convert)


In [12]:
def convert_cast(object):
  list = []
  counter = 0
  for i in ast.literal_eval(object) :
    if counter != 3:
      list.append(i['name'])
      counter+=1
    else:
      break
  return list

In [13]:
#for cast take top three actors in the movie
movies['cast'] = movies['cast'].apply(convert_cast)


In [14]:
def convert_fetch_director(object):
  list = []
  counter = 0
  for i in ast.literal_eval(object) :
    if i['job'] == 'Director':
      list.append(i['name'])
      break
  return list

In [15]:
#for crew, extract dictionary with 'job':'Director'
movies['crew'] = movies['crew'].apply(convert_fetch_director)

In [16]:
#convert overview from string to list
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [17]:
#before merging all the tag columns, we need to remove spaces between the names so that the movies are appropriately tagged
# Sam A and Sam B are changed to SamA and SamB, or else the tages would be 'Sam','A','B' and 'Sam' tag will be in two movies, model will be
# confused about which sam you want 
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [18]:
#mergins to form tags
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [19]:
#creating new dataframe with just id, title and tags
new_movies = movies[['movie_id','title','tags']]

In [20]:
#joing all the tags to form a long string 
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))

C:\Users\Anna Thomas\AppData\Local\Temp\ipykernel_1600\1560101202.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))


In [ ]:
new_movies['tags'][0]

In [21]:
#converting the tags to lowercase
new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())

C:\Users\Anna Thomas\AppData\Local\Temp\ipykernel_1600\2186437100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(lambda x:x.lower())


In [22]:
#Text Vectorization
#now we have to convert the text tags into vectors in a way that each movie becomes a vector 
#similar movies have their vectors close to eachother 
# we are using bag of words
#concat all tags and find the K most common words 
#remove stop words - or, and, from etc before vectorization

#we use sklearn CountVectorizer 
cv = CountVectorizer(max_features = 5000,stop_words = 'english')


In [23]:
#converting to matrix to count frequency of each word in a movie
vectors = cv.fit_transform(new_movies['tags']).toarray()

In [ ]:
#5000 most frequency words 
cv.get_feature_names()

In [24]:
#remove duplicates like 'actor' and 'actors'
# we apply stemming : ['actor'.'actors'] => ['actor','actor']
# we use nltk library
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [25]:
def stem(text):
  y = []
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y)

In [26]:
new_movies['tags'] = new_movies['tags'].apply(stem)

C:\Users\Anna Thomas\AppData\Local\Temp\ipykernel_1600\3229886743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_movies['tags'] = new_movies['tags'].apply(stem)


In [27]:
#each movie is a vector now
# we need to calculate disance between each vector
# smaller the distance, greater the similarity
# We will calculate cosine distance - consider the two vectors and find the angle between them
# smaller the angle, greater the cosine value 
#euclidean distance fails in higher dimensions, so we use cosine distance 

similarity_matrix = cosine_similarity(vectors)
# matrix having similarity between every movie to everyother movie

In [28]:
def recommend(movie):
  #fetching movie index
  movie_index = new_movies[new_movies['title'] == movie].index[0]
  distances = similarity_matrix[movie_index]
  #enumerate: to preserve the index when we reverse sort to find top 5 similar movies
  #sorting based on 1 ie similarity   
  movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

  for i in movies_list:
    print(new_movies.iloc[i[0]].title)


In [29]:
recommend('Alice in Wonderland')

Alice Through the Looking Glass
Mirrormask
Spirited Away
Charlie and the Chocolate Factory
The Chronicles of Narnia: The Voyage of the Dawn Treader


In [32]:
#to send the top 5 similar mvoies to the app
pickle.dump(new_movies.to_dict(), open('movies_dict.pkl','wb'))

In [33]:
pickle.dump(similarity_matrix, open('movies_similarity.pkl','wb'))

In [ ]:
new_movies.head()